In [1]:
import sys
# setting path
sys.path.append('..')

In [2]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt

from src import io, var, utils, fourier, lin_reg, reconstruction
from vis import plotter

import importlib
importlib.reload(io)
importlib.reload(var)
importlib.reload(utils)
importlib.reload(fourier)
importlib.reload(lin_reg)
importlib.reload(reconstruction)

importlib.reload(plotter)

<module 'vis.plotter' from '/home/ray/git-projects/spec_appx/notebooks/../vis/plotter.py'>

Prepare orography by generating underlying lat-lon grid of interest.

In [3]:
lat_min = 48.0
lat_max = 64.0

lon_min = -148.0
lon_max = -112.0

# dlat, dlon in degs
dlat = 0.05
dlon = 0.05

lat = np.arange(lat_min - dlat, lat_max + dlat, dlat)
lon = np.arange(lon_min - dlon, lon_max + dlon, dlon)

lat = np.deg2rad(lat)
lon = np.deg2rad(lon)

lat_mgrid, lon_mgrid = np.meshgrid(lat,lon)

In [4]:
grid = var.grid()
    
reader = io.ncdata()
fn = '../data/icon_grid_0012_R02B04_G_linked.nc'
reader.read_dat(fn, grid)
# grid.apply_f(utils.rad2deg)

vids = []
for lat_ref in lat:
    for lon_ref in lon:
        vid = utils.pick_cell(lat_ref, lon_ref, grid)
        vids.append(vid)


In [5]:
icon_cell_indexes = np.array(list((set(vids))))

clat = grid.clat
clat_verts = grid.clat_vertices
clon = grid.clon
clon_verts = grid.clon_vertices
links = grid.links

print(clat)
print(links)

[ 1.55152168  1.52214143  1.53717002 ... -0.48917738 -0.49075103
 -0.47920845]
[[75 79 83 ...  0  0  0]
 [75 79  0 ...  0  0  0]
 [75 79 83 ...  0  0  0]
 ...
 [26 50  0 ...  0  0  0]
 [26 50  0 ...  0  0  0]
 [26 50  0 ...  0  0  0]]


In [6]:
links[icon_cell_indexes].shape

(184, 108)

In [7]:
comp_clat = clat[icon_cell_indexes]
comp_clon = clon[icon_cell_indexes]
comp_clat_verts = clat_verts[icon_cell_indexes]
comp_clon_verts = clon_verts[icon_cell_indexes]
comp_links = links[icon_cell_indexes]

sorted_unique_links = np.sort(list(set(comp_links[np.where(comp_links > 0)])))
print(sorted_unique_links)

for new_id, link_id in enumerate(sorted_unique_links):
    comp_links[np.where(comp_links == link_id)] = new_id + 1

print(comp_links)

[40 44 47 64 68 71 92]
[[2 5 0 ... 0 0 0]
 [2 5 0 ... 0 0 0]
 [2 5 0 ... 0 0 0]
 ...
 [1 2 4 ... 0 0 0]
 [1 2 4 ... 0 0 0]
 [1 4 0 ... 0 0 0]]


In [8]:
ncfile = nc.Dataset('../data/icon_compact.nc',mode='w') 
print(ncfile)

cell = ncfile.createDimension('cell', np.size(comp_clat))     # latitude axis
nv = ncfile.createDimension('nv', 3)    # longitude axis
nlinks = ncfile.createDimension('nlinks', links.shape[1])    # link length
for dim in ncfile.dimensions.items():
    print(dim)

ncfile.title='Compact ICON grid for testing and debugging purposes'
print(ncfile.title)

clat = ncfile.createVariable('clat', np.float32, ('cell',))
clat.units = 'radian'
clat.long_name = 'center latitude'

clon = ncfile.createVariable('clon', np.float32, ('cell',))
clon.units = 'radian'
clon.long_name = 'center longitude'

clat_verts = ncfile.createVariable('clat_vertices', np.float32, ('cell','nv',))
clat_verts.units = 'radian'

clon_verts = ncfile.createVariable('clon_vertices', np.float32, ('cell','nv',))
clon_verts.units = 'radian'

clinks = ncfile.createVariable('links', np.int32, ('cell','nlinks',))
clinks.units = ''

clat[:] = comp_clat
clon[:] = comp_clon
clat_verts[:,:] = comp_clat_verts
clon_verts[:,:] = comp_clon_verts
clinks[:,:] = comp_links

print(clon_verts[:,:])

ncfile.close(); print('Dataset is closed!')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): 
    variables(dimensions): 
    groups: 
('cell', <class 'netCDF4._netCDF4.Dimension'>: name = 'cell', size = 184)
('nv', <class 'netCDF4._netCDF4.Dimension'>: name = 'nv', size = 3)
('nlinks', <class 'netCDF4._netCDF4.Dimension'>: name = 'nlinks', size = 108)
Compact ICON grid for testing and debugging purposes
[[-2.2920494 -2.2536633 -2.3105268]
 [-2.2536633 -2.212061  -2.2710307]
 [-2.2710307 -2.3105268 -2.2536633]
 [-2.3105268 -2.2710307 -2.3308356]
 [-2.212061  -2.1670187 -2.2280848]
 [-2.1670187 -2.1183457 -2.181394 ]
 [-2.181394  -2.2280848 -2.1670187]
 [-2.2280848 -2.181394  -2.245909 ]
 [-2.245909  -2.2902462 -2.2280848]
 [-2.2902462 -2.3308356 -2.2710307]
 [-2.2710307 -2.2280848 -2.2902462]
 [-2.2280848 -2.2710307 -2.212061 ]
 [-2.3308356 -2.2902462 -2.3533208]
 [-2.2902462 -2.245909  -2.3116412]
 [-2.3116412 -2.3533208 -2.2902462]
 [-2.3533208 -2.3116412 -2.3783152]


In [9]:
links_tmp = links[icon_cell_indexes].flatten()
links_tmp = links_tmp[np.where(links_tmp > 0)]
links_tmp = np.sort(list(set(links_tmp)))
links_tmp -= 1

print(links_tmp)

topo = var.topo()
fn = '../data/GMTED2010_topoGlobal_SGS_30ArcSec.nc'
reader.read_dat(fn, topo)

lon = topo.lon
lat = topo.lat
z = topo.topo

del topo

# lon, lat, z = readnc(datfile, var)
nrecords = np.shape(z)[0]; nlon = np.shape(lon)[1]; nlat = np.shape(lat)[1]

sz_tmp = np.size(links_tmp)
# print(nlat,nlon, np.size(links_tmp))

compactified_topo = np.zeros((sz_tmp,nlat, nlon))
print(compactified_topo.shape)

compactified_lat = np.zeros((sz_tmp,nlat))
compactified_lon = np.zeros((sz_tmp,nlon))
    
for i,lnk in enumerate(links_tmp):
    print("i, lnk = ", (i, lnk))
    compactified_lat[i] = lat[lnk]
    compactified_lon[i] = lon[lnk]
    compactified_topo[i] = z[lnk]
    
del lat, lon, z

ncfile = nc.Dataset('../data/topo_compact.nc',mode='w',format='NETCDF4_CLASSIC') 
print(ncfile)

nfiles = ncfile.createDimension('nfiles', sz_tmp)
lat = ncfile.createDimension('lat', nlat)
lon = ncfile.createDimension('lon', nlon)
for dim in ncfile.dimensions.items():
    print(dim)

ncfile.title='Compact GMTED2010 USGS Topography grid for testing and debugging purposes'
print(ncfile.title)

lat = ncfile.createVariable('lat', np.float32, ('nfiles','lat'))
lat.units = 'degrees'

lon = ncfile.createVariable('lon', np.float32, ('nfiles','lon'))
lon.units = 'degrees'

topo = ncfile.createVariable('topo', np.float32, ('nfiles','lat','lon'))
topo.units = 'meters'

lat[:,:] = compactified_lat
lon[:,:] = compactified_lon
topo[:,:,:] = compactified_topo

ncfile.close(); print('Dataset is closed!')

[39 43 46 63 67 70 91]
(7, 2400, 3600)
i, lnk =  (0, 39)
i, lnk =  (1, 43)
i, lnk =  (2, 46)
i, lnk =  (3, 63)
i, lnk =  (4, 67)
i, lnk =  (5, 70)
i, lnk =  (6, 91)
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    dimensions(sizes): 
    variables(dimensions): 
    groups: 
('nfiles', <class 'netCDF4._netCDF4.Dimension'>: name = 'nfiles', size = 7)
('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 2400)
('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 3600)
Compact GMTED2010 USGS Topography grid for testing and debugging purposes
Dataset is closed!
